#**Automated Plant Leaf Disease Detection (APLDD)**
Project Group 25 Members:
*   Fabin Joe: 1005889709
*   Kshitij Bhardwaj:
*   Rahul Chopra:
*   Vishal Radhakrishnan:

Neural Network to analyze images of a plant leaf to identify various common diseases. Our project is indented to be used in the **Agricultural Sector** to result in early detection of diseases.

#Some Coding Guidelines so our code seems uniform
Class Definaitions: class **HappyPerson**(): 

Function Definations: def **makePersonHappy**():

Variable Definations: **measure_happiness** = 0


In [5]:
# All the import stuff
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import os
import shutil
import pandas as pd

# Data Cleaning

In [1]:
# Function to Resize a given image
# - Vishal

2


In [7]:
# Function to organize imgs into directory
# - Fabin
def changeImgDirectory(old_directory, new_directory):
    if not os.path.exists(new_directory):
        os.mkdir( new_directory )

    for sub_direcotry in os.listdir(old_directory):
        shutil.move( old_directory + "\\" + sub_direcotry, new_directory )

    print( f"Copied Directory {old_directory} to {new_directory}" )

def unsqueezeImgDirectory(path, directory):
    for sub_directory in os.listdir(path + "\\" + directory):
        new_sub_directory = sub_directory.title()
        new_sub_directory = sub_directory.replace( " ", "" )
        new_sub_directory = directory + "_" + new_sub_directory

        new_directory = path + "\\" + new_sub_directory
        shutil.move(path + "\\" + directory + "\\" + sub_directory, new_directory)
        os.rmdir(path + "\\" + directory)
    print( f"Unsqueezed Directory {directory}" )

def createDirectoryFromCSV(path, directory, path_to_csv):
    data = pd.read_excel(path_to_csv)

    category_to_label = {}
    for index in range(len(data)):
        entry = data.iloc[index]

        img_name = entry.iloc[0]
        category = entry.iloc[1]

        old_img_path = path + "\\" + directory + "\\" + img_name
        
        if category not in category_to_label:
            label = category.replace("_", " ")
            label = label.title()
            label = label.replace(" ", "")
            
            category_to_label[category] = label
            new_img_path = path + "\\" + label

            os.mkdir(new_img_path)
        
        new_img_path = path + "\\" + category_to_label[category] + "\\" + img_name

        shutil.move( old_img_path, new_img_path )
    print( "Images segmented into directories" )
    pass

In [77]:
os.mkdir(r"Dataset")
changeImgDirectory(r"Citrus Plant Dataset\Leaves", r"Dataset\Citrus")
changeImgDirectory(r"Coffee Plant Dataset\Photos", r"Dataset\Coffee")
changeImgDirectory(r"Pear Dataset\Leaves", r"Dataset\Pear")
changeImgDirectory(r"plantvillage dataset\color", r"Dataset")

Copied Directory Citrus Plant Dataset\Leaves to Dataset\Citrus
Copied Directory Coffee Plant Dataset\Photos to Dataset\Coffee
Copied Directory Pear Dataset\Leaves to Dataset\Pear
Copied Directory plantvillage dataset\color to Dataset


In [99]:
unsqueezeImgDirectory(r"Dataset", r"Citrus")
unsqueezeImgDirectory(r"Dataset", r"Pear")

Unsqueezed Directory Citrus
Unsqueezed Directory Pear


In [8]:
PATH = r"Dataset"
directory = r"Coffee"
path_to_csv = r"Coffee Plant Dataset\Annotations\RoCoLE-classes.xlsx"
createDirectoryFromCSV(PATH, directory, path_to_csv)

Images segmented into directories


In [9]:
PATH = r"Dataset\Coffee"

print("Hello")

Hello


In [24]:
# Function to Create the csv file
# - Fabin
def generateCSV(dataset_path):

    d = {'Image_Path': [], 'Label': [], 'Plant': [], 'Disease': [] }
    data = pd.DataFrame(data=d)

    for label in os.listdir(dataset_path):
        label_path = dataset_path + "\\" + label
        for img_name in os.listdir(label_path):
            img_path = label_path + "\\" + img_name
            data = data.append( {'Image_Path': img_path, 'Label': label, 'Plant': "", 'Disease': "" }, ignore_index = True )            
        print( f"Label {label} is added to the CSV " )
    
    dataset_csv_path = 'dataset_csv.csv'
    data.to_csv(dataset_csv_path, index=False)
    print( f"CSV file is generated and stored at {dataset_csv_path}" )

    return dataset_csv_path

In [25]:
dataset_path = r"Dataset"
dataset_csv_path = generateCSV(dataset_path)

Label Apple_AppleScab is added to the CSV 
Label Apple_BlackRot is added to the CSV 
Label Apple_CedarAppleRust is added to the CSV 
Label Apple_Healthy is added to the CSV 
Label Blueberry_Healthy is added to the CSV 
Label Cherry_Healthy is added to the CSV 
Label Cherry_PowderyMildew is added to the CSV 
Label Citrus_BlackSpot is added to the CSV 
Label Citrus_Canker is added to the CSV 
Label Citrus_Greening is added to the CSV 
Label Citrus_Healthy is added to the CSV 
Label Citrus_Melanose is added to the CSV 
Label Coffee_Healthy is added to the CSV 
Label Coffee_RedSpiderMite is added to the CSV 
Label Coffee_RustLevel1 is added to the CSV 
Label Coffee_RustLevel2 is added to the CSV 
Label Coffee_RustLevel3 is added to the CSV 
Label Coffee_RustLevel4 is added to the CSV 
Label Corn_CercosporaLeafSpot is added to the CSV 
Label Corn_CommonRust is added to the CSV 
Label Corn_Healthy is added to the CSV 
Label Corn_NorthernLeafBlight is added to the CSV 
Label Grape_BlackRot is

In [28]:
total = 0
for label in os.listdir("Dataset"):
    path = "Dataset\\" + label
    length = len(os.listdir(path))
    print( f" {label} : {length}" )
    total += length
print(total)



 Apple_AppleScab : 630
 Apple_BlackRot : 621
 Apple_CedarAppleRust : 275
 Apple_Healthy : 1645
 Blueberry_Healthy : 1502
 Cherry_Healthy : 854
 Cherry_PowderyMildew : 1052
 Citrus_BlackSpot : 171
 Citrus_Canker : 163
 Citrus_Greening : 204
 Citrus_Healthy : 58
 Citrus_Melanose : 13
 Coffee_Healthy : 791
 Coffee_RedSpiderMite : 167
 Coffee_RustLevel1 : 344
 Coffee_RustLevel2 : 166
 Coffee_RustLevel3 : 62
 Coffee_RustLevel4 : 30
 Corn_CercosporaLeafSpot : 513
 Corn_CommonRust : 1192
 Corn_Healthy : 1162
 Corn_NorthernLeafBlight : 985
 Grape_BlackRot : 1180
 Grape_Esca : 1383
 Grape_Healthy : 423
 Grape_LeafBlight : 1076
 Orange_Haunglongbing : 5507
 Peach_BacterialSpot : 2297
 Peach_Healthy : 360
 Pear_Curl : 65
 Pear_Healthy : 43
 Pear_Slug : 4050
 Pear_Spot : 1768
 PepperBell_BacterialSpot : 997
 PepperBell_Healthy : 1478
 Potato_EarlyBlight : 1000
 Potato_Healthy : 152
 Potato_LateBlight : 1000
 Raspberry_Healthy : 371
 Soybean_Healthy : 5090
 Squash_PowderyMildew : 1835
 Strawberry_H

In [ ]:
# Function to relable and fill csv file
# - Rahul

In [ ]:
# Function to cause a train val test split
# - Kshitij

In [ ]:
# Function to segment images - optional maybe we can do this later.

# Baseline Model

# Neural Network